In [1]:
import awkward as ak
import numpy as np
import time
import coffea
print(coffea.__version__)
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
files = ["root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/0CDD30E0-B953-E54B-BB7A-BFB45BE6EC93.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/F6B6A983-3E2B-7B4D-A2CE-E03AB6E32387.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/FDF7FBE4-9226-994A-BD45-29995A3CD62F.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/2CEB03D7-5B1A-EE46-9C21-13ED3E349AF4.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/DB87A517-0584-1A41-BB18-AAE7C9AAE5B8.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/BFE2E7E9-82CB-FE47-9B43-EBA5C373AF60.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/5213B536-BAA1-CC43-A58A-B1E41E2680F5.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/E8400F91-3216-EA46-A9F1-8CC7EF5261B0.root",
"root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root"]
#/store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/7CB92FEA-6DFF-7647-9D7A-094D1F84245B.root has timeout error

0.7.9


In [15]:
from coffea import processor, hist

class TriJetHists(processor.ProcessorABC):
    def __init__(self):
        dataset_cats = hist.Cat("dataset", "Dataset")
        jet_cats = hist.Cat("jetNumb", "Jet")
        parton_cats = hist.Cat("parton_flav", "Parton Flavour")
        mass_bins = hist.Bin("mass", "Jet Mass (GeV)", 60, 0, 120)
        pt_bins = hist.Bin("pt", "Jet pT (GeV)", 60, 0, 2400)
        btag_bins = hist.Bin("bdisc", "B-tag discriminator", 10, 0.0, 1.)
        self._histos = processor.dict_accumulator({
        'mass1':          hist.Hist("Events", jet_cats, mass_bins, ),
        'mass2':          hist.Hist("Events", jet_cats, mass_bins, ),
        'mass3':          hist.Hist("Events", jet_cats, mass_bins, ),
        'genmass1':       hist.Hist("Events", jet_cats, mass_bins, ),
        'genmass2':       hist.Hist("Events", jet_cats, mass_bins, ),
        'genmass3':       hist.Hist("Events", jet_cats, mass_bins, ),
        'pt3_g':        hist.Hist("Events", parton_cats, pt_bins, ),
        'pt3_uds':        hist.Hist("Events", parton_cats, pt_bins, ),
        'pt3_c':        hist.Hist("Events", parton_cats, pt_bins, ),
        'pt3_b':        hist.Hist("Events", parton_cats, pt_bins, ),
        'mass3_g':        hist.Hist("Events", parton_cats, mass_bins, ),
        'mass3_uds':      hist.Hist("Events", parton_cats, mass_bins, ),
        'mass3_c':        hist.Hist("Events", parton_cats, mass_bins, ),
        'mass3_b':        hist.Hist("Events", parton_cats, mass_bins, ),
        'weights':        hist.Hist("Events", dataset_cats, hist.Bin("weights", "gen weights", 60, 0, 1), ),
        'btag_b':         hist.Hist("Events", parton_cats, btag_bins),
        'btag_gluon':     hist.Hist("Events", parton_cats, btag_bins),
        'gluonPurity':    hist.Hist("Events", parton_cats, btag_bins),
        'nGluonJets': processor.defaultdict_accumulator(int),
        'cutflow': processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._histos
    
    # we will receive a NanoEvents instead of a coffea DataFrame
    def process(self, events):
        out = self.accumulator.identity()
        trijetevents = events[
            (ak.num(events.FatJet) >= 3) & (ak.num(events.GenJetAK8) >= 3)#add pt > 100 here???
        ]
        trijetevents = events[events.FatJet.pt >= 200. ]
        pt_cuts = [0, 50., 100., 200., 400., 800., 1600.]
        #get leading 3 jets
        jet1 = trijetevents.FatJet[:, 0]
        jet2 = trijetevents.FatJet[:, 1]
        jet3 = trijetevents.FatJet[:, 2]
        
#        print("Softest jets after >3 selection", len(jet3))
        
        
        #calculate dphi_min
        dphi12 = np.abs(jet1.delta_phi(jet2))
        dphi13 = np.abs(jet1.delta_phi(jet3))
        dphi23 = np.abs(jet2.delta_phi(jet3))
        
        dphi_min = np.amin([dphi12, dphi13, dphi23], axis = 0)

        #do the same for gen
        
        #do same for gen; jets might not be same order reco and gen
        genjet1 = trijetevents.GenJetAK8[:,0]
        genjet2 = trijetevents.GenJetAK8[:,1]
        genjet3 = trijetevents.GenJetAK8[:,2]
        
        dphi12_gen = np.abs(genjet1.delta_phi(genjet2))
        dphi13_gen = np.abs(genjet1.delta_phi(genjet3))
        dphi23_gen = np.abs(genjet2.delta_phi(genjet3))
        
        dphimin_gen = np.amin([dphi12_gen, dphi13_gen, dphi23_gen], axis = 0)
        
        #apply dphi gen and reco selection
        trijetevents = trijetevents[(dphimin_gen > 1.0) & (dphi_min > 1.0)]
        #match jets
        matched1 = ~ak.is_none(trijetevents.FatJet[:,0].matched_gen)
        matched2 = ~ak.is_none(trijetevents.FatJet[:,1].matched_gen)
        matched3 = ~ak.is_none(trijetevents.FatJet[:,2].matched_gen)
        fake_jets = trijetevents.FatJet[ak.is_none(trijetevents.FatJet.matched_gen)]
        jet1 = trijetevents.FatJet[:,0][matched1]
        jet2 = trijetevents.FatJet[:,1][matched2]
        jet3 = trijetevents.FatJet[:,2][matched3]
        
        genjet1 = jet1.matched_gen
        genjet2 = jet2.matched_gen
        genjet3 = jet3.matched_gen

#        print("Softest jets after dphi selection and matching", len(jet3))
        
        #flavour --> 21 is gluon
        jet3_g = jet3[np.abs(genjet3.partonFlavour) == 21]
        jet3_uds = jet3[np.abs(genjet3.partonFlavour) < 4]
        jet3_c = jet3[np.abs(genjet3.partonFlavour) == 4]
        jet3_b = jet3[np.abs(genjet3.partonFlavour) == 5]
        
        print("Gluon purity of third jet for basic selection: ", len(jet3_g)/len(jet3))
        
        #now require leading two jets b-tagged w/ loose discr.
        trijetevents_bbloose = trijetevents[(trijetevents.FatJet[:,0].btagCSVV2 >= 0.460) & (trijetevents.FatJet[:,1].btagCSVV2 >= 0.460)]
        matched3_btag = ~ak.is_none(trijetevents_bbloose.FatJet[:,2].matched_gen)
        jet3_bbloose = trijetevents_bbloose.FatJet[:,2][matched3_btag]
        genjet3_bbloose = jet3_bbloose.matched_gen
        gPurity_bbloose = len(jet3_bbloose[np.abs(jet3_bbloose.matched_gen.partonFlavour) == 21])/len(jet3_bbloose)
        #NEED TO ACCUMULATE THESE NUMBERS
    
        #now require leading two jets b-tagged w/ medium discr.
        trijetevents_bbmed = trijetevents[(trijetevents.FatJet[:,0].btagCSVV2 >= 0.80) & (trijetevents.FatJet[:,1].btagCSVV2 >= 0.80)]
        matched3_btag = ~ak.is_none(trijetevents_bbmed.FatJet[:,2].matched_gen)
        jet3_bbmed = trijetevents_bbmed.FatJet[:,2][matched3_btag]
        genjet3_bbmed = jet3_bbmed.matched_gen
        gPurity_bbmed = len(jet3_bbmed[np.abs(genjet3_bbmed.partonFlavour) == 21])/len(jet3_bbmed)
        print("Number of gluon jets after bb med selection", len(jet3_bbmed[np.abs(genjet3_bbmed.partonFlavour) == 21]))
                         
        #require only leading jet btag loose
        #now require leading two jets b-tagged w/ loose discr.
        trijetevents_bloose = trijetevents[(trijetevents.FatJet[:,0].btagCSVV2 >= 0.460)]
        matched3_btag = ~ak.is_none(trijetevents_bloose.FatJet[:,2].matched_gen)
        jet3_bloose = trijetevents_bloose.FatJet[:,2][matched3_btag]
        genjet3_bloose = jet3_bloose.matched_gen
        gPurity_bloose = len(jet3_bloose[np.abs(jet3_bloose.matched_gen.partonFlavour) == 21])/len(jet3_bloose)
        
        
        out['mass1'].fill(
            jetNumb = "Leading Jet",
            mass=jet1.mass,
            weight=trijetevents.Generator.weight[matched1]
        )
        out['mass2'].fill(
            jetNumb = "Jet2",
            mass=jet2.mass,
            weight=trijetevents.Generator.weight[matched2]
        )
        out['mass3'].fill(
            jetNumb = "Jet3",
            mass=jet3.mass,
            weight=trijetevents.Generator.weight[matched3]
        )
        #NOTE --> need gen sd mass eventually --> recluster :( 
        out['genmass1'].fill(
            jetNumb = "Leading Gen Jet",
            mass=genjet1.mass,
            weight=trijetevents.Generator.weight[~ak.is_none(trijetevents.FatJet[:,0].matched_gen)]
        )
        out['genmass2'].fill(
            jetNumb = "Gen Jet2",
            mass=genjet2.mass,
            weight=trijetevents.Generator.weight[matched2]
        )
        out['genmass3'].fill(
            jetNumb = "Gen Jet 3",
            mass=genjet3.mass,
            weight=trijetevents.Generator.weight[matched3]
        )
        out['mass3_g'].fill(
            parton_flav = "Gluon",
            mass=jet3_g.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 21]
        )
        out['mass3_uds'].fill(
            parton_flav = "UDS",
            mass=jet3_uds.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) < 4]
        )
        out['mass3_c'].fill(
            parton_flav = "Charm",
            mass=jet3_c.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 4]
        )
        out['mass3_b'].fill(
            parton_flav = "Bottom",
            mass=jet3_b.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 5]
        )
        out['pt3_g'].fill(
            parton_flav = "Gluon",
            pt=jet3_g.pt,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 21]
        )
        out['pt3_b'].fill(
            parton_flav = "Bottom",
            pt=jet3_b.pt,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 5]
        )
        out['pt3_uds'].fill(
            parton_flav = "UDS",
            pt=jet3_uds.pt,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) < 4]
        )
        out['pt3_c'].fill(
            parton_flav = "Charm",
            pt=jet3_c.pt,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 4]
        )
        out['weights'].fill(
            dataset=events.metadata["dataset"],
            weights=trijetevents.Generator.weight,
        )     
        out['btag_gluon'].fill(
            parton_flav = "Gluon",
            bdisc=jet3_g.btagCSVV2,
        )
        out['btag_b'].fill(
            parton_flav = "Bottom",
            bdisc=jet3_b.btagCSVV2,
        )
        out['cutflow']['chunks'] += 1
        out['cutflow']['gluonPuritySum_bloose'] += gPurity_bloose
        out['cutflow']['gluonPuritySum_bbmed'] += gPurity_bbmed
        out['cutflow']['gluonPuritySum_bbloose'] += gPurity_bbloose
        out['cutflow']['nGluonJets'] += (len(jet3[np.abs(genjet3.partonFlavour) == 21]))
        out['cutflow']['nGluonJets_bbloose'] += (len(jet3_bbloose[np.abs(genjet3_bbloose.partonFlavour) == 21]))
        out['cutflow']['nGluonJets_bloose'] += (len(jet3_bloose[np.abs(genjet3_bloose.partonFlavour) == 21]))
        return out
    
    def postprocess(self, accumulator):
        return accumulator

In [ ]:

samples = {
    "QCD": files
}

result = processor.run_uproot_job(
    samples,
    "Events",
    TriJetHists(),
    processor.futures_executor,
    {"schema": NanoAODSchema},
)

Processing:   0%|          | 0/45 [00:00<?, ?chunk/s]

Gluon purity of third jet for basic selection:  0.5690721649484536
Number of gluon jets after bb med selection 86
Gluon purity of third jet for basic selection:  0.5529615218331172
Number of gluon jets after bb med selection 83
Gluon purity of third jet for basic selection:  0.528548363422868
Number of gluon jets after bb med selection 334
Gluon purity of third jet for basic selection:  0.5187105816398038
Number of gluon jets after bb med selection 263
Gluon purity of third jet for basic selection:  0.5132363132363132
Number of gluon jets after bb med selection 269
Gluon purity of third jet for basic selection:  0.5125161115256767
Number of gluon jets after bb med selection 274
Gluon purity of third jet for basic selection:  0.5169235658668362
Number of gluon jets after bb med selection 150
Gluon purity of third jet for basic selection:  0.5101704233095107
Number of gluon jets after bb med selection 152
Gluon purity of third jet for basic selection:  0.5141990196841204
Number of gluon 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
hist.plot1d(result['mass3_g'])
hist.plot1d(result['mass3_uds'])
hist.plot1d(result['mass3_b'])
hist.plot1d(result['mass3_c'])
ax.set_xlabel("Jet3 mass (GeV)")

In [ ]:
fig, ax = plt.subplots(1, 1)
hist.plot1d(result['pt3_g'])
hist.plot1d(result['pt3_uds'])
hist.plot1d(result['pt3_b'])
hist.plot1d(result['pt3_c'])
ax.set_xlabel("Jet3 pT (GeV)")

In [ ]:
hist.plot1d(result['mass3'])
hist.plot1d(result['genmass3'])

In [ ]:
fig, ax = plt.subplots(1, 1)
hist.plot1d(result['btag_b'])
hist.plot1d(result['btag_gluon'])
ax.set_xlabel("Jet3 b-tag")
ax.set_ylim([0, 60000])

In [ ]:
print(result['cutflow']['gluonPuritySum_bloose']/result['cutflow']['chunks'])
print(result['cutflow']['gluonPuritySum_bbmed']/result['cutflow']['chunks'])
print(result['cutflow']['gluonPuritySum_bbloose']/result['cutflow']['chunks'])

In [ ]:
print(result['cutflow']['nGluonJets'])
print(result['cutflow']['nGluonJets_bloose'])

In [9]:
print(events.FatJet.fields)
print(events.Generator.fields)

NameError: name 'events' is not defined

In [ ]:
#plot pt, eta, phi, fake rate, pt efficiency, gluon purity for difference selections with b tags for all jets 
#and each individual jet in the 3 jets